In [28]:
import MeCab
import mojimoji


class MecabTokenizer:
    def __init__(self):
        self.tagger = MeCab.Tagger("-u dict/custom.dic")
        # Note: node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        self.tagger.parse('')

    def tokenize(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.tokenize_single_text(text))
        return splited_texts

    def tokenize_single_text(self, text):
        node = self.tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(',')
            pos = features[0]

            if pos in ["名詞", "動詞", "形容詞", "感動詞", "助動詞", "副詞"]:
                lemma = node.feature.split(",")[6]

                if pos == '名詞' and features[1] == '非自立':
                    node = node.next
                    continue
                if pos == '動詞' and features[1] == '非自立':
                    node = node.next
                    continue

                if pos == '助動詞' and lemma != 'ない':
                    node = node.next
                    continue

                if lemma == 'ある':
                    node = node.next
                    continue

                if lemma == "*":
                    lemma = node.surface

                word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return word_list

In [29]:
import pandas as pd

data = pd.read_table('/Users/shwld/Downloads/wiki-corpus/jawiki.txt')

In [30]:
from sklearn.externals import joblib

In [31]:
tokenized_sentences = MecabTokenizer().tokenize(data['question'])
joblib.dump(tokenized_sentences, '/Users/shwld/Downloads/wiki-corpus/tokenized_sentences_list')

SystemError: <built-in function Tagger_parseToNode> returned a result with an error set

In [ ]:
# tokenized_sentences = joblib.load('/Users/shwld/Downloads/wiki-corpus/tokenized_sentences_list')

In [ ]:
from gensim import models
from gensim import corpora, models, similarities

In [ ]:
# tokenized_sentences = [
#     ['アンパサンド', '（，', '＆）', '意味', 'する', '記号', 'ラテン語', '合', '字', 'Ｔｒｅｂｕｃｈｅｔ', 'ＭＳ', 'フォント', '表示', 'する', 'れる', '”', 'ｅｔ', '”', '合', '字', '容易', 'わかる', 'ａｍｐｅｒｓａ', '”', 'ａｎｄ', 'ｐｅｒ', 'ｓｅ', 'ａｎｄ', '”、', '意味', '”', 'ａｎｄ', '［', 'ｔｈｅ', 'ｓｙｍｂｏｌ', 'ｗｈｉｃｈ', '］', 'ｂｙ', 'ｉｔｓｅｌｆ', '［', 'ｉｓ', '］', 'ａｎｄ', '”'],
#     ['使用', '１', '世紀', '遡る', 'できる', '（', '１', '）、', '５', '世紀', '中葉', '（', '２', '，', '３', '）', '現代', '（', '４', '−', '６', '）', '至る', '変遷', 'わかる'],
#     ['Ｚ', '続く', 'ラテン', '文字', 'アルファベット', '２７', '字', '目', 'する', 'れる', '時期'],
#     ['アンパサンド', '役割', '果たす', '文字', 'ｅｔ', '呼ぶ', 'れる', '数字', '７', '似る', '記号', '（，', 'Ｕ', '＋', '２０４', 'Ａ', '）。', '記号', '現在', 'ゲール', '文字', '使う', 'れる'],
#     ['記号', '名', 'アンパサンド', 'ラテン語', 'まじる', '英語', '「＆', 'それ', '自身', '”', 'ａｎｄ', '”', '表す', '（＆', 'ｐｅｒ', 'ｓｅ', 'ａｎｄ', '）', 'くずれる', '形', '英語', '言語', '名称', '多様'],
#     ['日常', '的', '手書き', '場合', '欧米', 'アンパサンド', '縦', '線', '引く', '単純', '化', 'する', 'れる', '使う', 'れる'],
#     ['同様', 'ｔ', '「＋（', 'プラス', '輪', '重ねる', '無声', '歯茎', '側面', '摩擦音', '示す', '発音', '記号', '使う', 'れる'],
#     ['プログラミング', '言語', 'Ｃ', '多数', '言語', 'ＡＮＤ', '演算', '子', '用いる', 'られる', 'Ｃ', '例'],
#     ['ＰＨＰ', '変数', '宣言', '記号', '（＄）', '直前', '記述', 'する', '参照', '渡し', '行う', 'できる'],
#     ['ＢＡＳＩＣ', '系列', '言語', '文字', '列', '連結', '演算', '子', '使用', 'する', 'れる', 'ｃｏｄｉｃｅ', '＿', '４', 'ｃｏｄｉｃｅ', '＿', '５', '返す', '主', 'マイクロソフト', '系', '整数', '十', '六', '進', '表記', 'ｃｏｄｉｃｅ', '＿', '６', '用いる', 'ｃｏｄｉｃｅ', '＿', '７', '十', '進', '１５', '表現', 'する']
# ]

In [18]:
gensim_dictionary=corpora.Dictionary(tokenized_sentences)
gensim_dictionary.save('/Users/shwld/Downloads/wiki-corpus/gensim.dict')

In [19]:
#corpusをid表示に変換する．二次元リストで構築しているので内包表記で記述
id_corpus=[gensim_dictionary.doc2bow(sentence) for sentence in tokenized_sentences];
#保存しておく 
corpora.MmCorpus.serialize('/Users/shwld/Downloads/wiki-corpus/gensim.mm', id_corpus);

In [20]:
#tfidfモデルの構築とcorpusの変換
tfidf=models.TfidfModel(id_corpus)
tfidf.save('/Users/shwld/Downloads/wiki-corpus/gensim.tfidf')
tfidf_corpus=tfidf[id_corpus]

In [21]:
#lsiモデルの構築とcorpusの変換
lsi=models.LsiModel(corpus=tfidf_corpus, num_topics=10, id2word=gensim_dictionary)
lsi.save('/Users/shwld/Downloads/wiki-corpus/gensim.lsi')
lsi_corpus=lsi[tfidf_corpus]

In [22]:
len(lsi_corpus)

10

In [23]:
queries = ['マイクロソフト', '演算']
query_vector = gensim_dictionary.doc2bow(queries)

In [24]:
vec_lsi = lsi[query_vector]
vec_lsi

[(0, 0.03736972832018761),
 (1, 0.14905692857376651),
 (2, -0.21773407396425185),
 (3, -0.097940788364416956),
 (4, 0.027534233926046335),
 (5, 0.045439409410188321),
 (6, 0.0024892799939609366),
 (7, 0.0034500561036447455),
 (8, 0.045117513174053991),
 (9, 0.016358396992505981)]

In [25]:
lsi_index = similarities.SparseMatrixSimilarity(lsi_corpus, num_features=len(lsi_corpus))
lsi_index.save('/Users/shwld/Downloads/wiki-corpus/gensim.lsi')

In [27]:
sims = lsi_index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims
# print(sorted(enumerate(sims), key=lambda item: -item[1])[:10])

[(9, 0.76265842),
 (7, 0.70202541),
 (2, 1.5832484e-08),
 (6, 6.9849193e-09),
 (5, 5.4205884e-09),
 (0, -7.4505806e-09),
 (3, -1.0011718e-08),
 (1, -1.4668331e-08),
 (8, -1.5599653e-08),
 (4, -2.2351742e-08)]

参照元
- http://kensuke-mi.hatenablog.com/entry/20131021/1382384297
- gensimでcosine_similarity
  - http://blog.yuku-t.com/entry/20110623/1308810518